# Import software libraries

In [ ]:
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

# Load a CSV file as a `DataFrame`

In [ ]:
complaints_data = pd.read_csv('/home/jovyan/work/ETL/data/consumer_loan_complaints.csv')

# Preview the first three rows of the data

In [ ]:
complaints_data.head(n = 3)

# Create a connection to the SQLite database

In [ ]:
conn = sqlite3.connect('/home/jovyan/work/ETL/data/user_data.db')
conn

# Read the `users` data

In [ ]:
# Write a query that selects everything from the users table.

query = 'SELECT * FROM users'

In [ ]:
# Read the query into a DataFrame.

users = pd.read_sql(query, conn)

# Preview the data.

users.head()

In [ ]:
# Check the shape of the data.

users.shape

# Read the `device` data

In [ ]:
query = 'SELECT * FROM device'

device = pd.read_sql(query, conn)

device.head()

In [ ]:
device.shape

# Read the `transactions` data

In [ ]:
# Read the user transactions in the last 30 days. 

query = 'SELECT * FROM transactions'

transactions = pd.read_sql(query, conn)

transactions.head()

In [ ]:
transactions.shape

# Aggregate the `transactions` data

In [ ]:
# Aggregate data on the number of transactions and the total amount.

query = '''SELECT user_id, 
                  COUNT(*) AS number_transactions, 
                  SUM(amount_usd) AS total_amount_usd 
           FROM transactions 
           GROUP BY user_id'''

transactions_agg = pd.read_sql(query, conn)

transactions_agg.head()

In [ ]:
transactions_agg.shape

# Merge the `device` table with the `users` table

In [ ]:
# Do a left join, as all users in the users table are of interest.

query = '''SELECT left_table.*,           
                  right_table.device       
           FROM users AS left_table         
           LEFT JOIN device AS right_table   
             ON left_table.user_id = right_table.user_id'''

users_w_device = pd.read_sql(query, conn)

In [ ]:
users_w_device.head(n = 3)

In [ ]:
users_w_device.shape

# Close the database connection

In [ ]:
conn.close()

# Merge `users_w_device` with `transactions_agg`

In [ ]:
# Do a right join so users won't be lost.

users_w_devices_and_transactions = \
transactions_agg.merge(users_w_device,
                       on = 'user_id', how = 'right')

users_w_devices_and_transactions.head()

In [ ]:
# Make sure number of rows is equal to users_w_devices table.

users_w_devices_and_transactions.shape

# Identify data where `age` is greater than 150

In [ ]:
users_w_devices_and_transactions[users_w_devices_and_transactions.age > 150]

# Drop incorrect data

In [ ]:
users_cleaned = \
users_w_devices_and_transactions[users_w_devices_and_transactions.age < 150]

users_cleaned.shape

# Identify more potentially erroneous data

In [ ]:
# Compare age to device.

pd.crosstab(users_cleaned['age'], users_cleaned['device'])

# Identify data types that need correcting

In [ ]:
users_cleaned.info()

In [ ]:
users_cleaned.default.value_counts()

# Convert the relevant variables to a Boolean type

In [ ]:
users_cleaned_1 = users_cleaned.copy()  # Work with a new object.

users_cleaned_1.default = \
users_cleaned_1.default.map(dict(yes = 1, no = 0)).astype(bool)

users_cleaned_1.default.value_counts()

In [ ]:
# Do the same for the other Boolean variables.

bool_vars = ['housing', 'loan', 'term_deposit']

for var in bool_vars:
    users_cleaned_1[var] = \
    users_cleaned_1[var].map(dict(yes = 1, no = 0)).astype(bool)

    print(f'Converted {var} to Boolean.')

In [ ]:
users_cleaned_1.info()

# Convert `date_joined` to a datetime format

In [ ]:
users_cleaned_2 = users_cleaned_1.copy()  # Work with a new object.

users_cleaned_2['date_joined'] = \
pd.to_datetime(users_cleaned_2['date_joined'],
               format = '%Y-%m-%d')

In [ ]:
users_cleaned_2.info()

# Identify all duplicated data

In [ ]:
duplicated_data = \
users_cleaned_2[users_cleaned_2.duplicated(keep = False)]

print('Number of rows with duplicated data:',
      duplicated_data.shape[0])

In [ ]:
duplicated_data

# Remove the duplicated data

In [ ]:
users_cleaned_final = \
users_cleaned_2[~users_cleaned_2.duplicated()]

users_cleaned_final[users_cleaned_final['user_id'] == \
'cba59442-af3c-41d7-a39c-0f9bffba0660']

In [ ]:
users_cleaned_final.shape

# Load data into an SQL database

In [ ]:
conn = sqlite3.connect('users_data_cleaned.db')

users_cleaned_final.to_sql('users_cleaned_final',
                           conn,
                           if_exists = 'replace',
                           index = False)

# Confirm that data was loaded into the database

In [ ]:
query = 'SELECT * FROM users_cleaned_final'

pd.read_sql(query, conn).head()

# Close the database connection

In [ ]:
conn.close()

# Write the `DataFrame` as a pickle file

In [ ]:
users_cleaned_final.to_pickle('users_data_cleaned.pickle')

# Confirm that the data was written to the pickle file

In [ ]:
pd.read_pickle('users_data_cleaned.pickle').head()

In [ ]:
pd.read_pickle('users_data_cleaned.pickle').info()

# Write the data to a CSV file

In [ ]:
users_cleaned_final.to_csv('users_data_cleaned.csv',
                           index = False)

# Confirm that the data was written to a CSV file.

In [ ]:
pd.read_csv('users_data_cleaned.csv').head()

In [ ]:
pd.read_csv('users_data_cleaned.csv').info()